In [1]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import pandas as pd
import numpy as np
import math
import orjson

import timeit
import random

In [2]:
start = timeit.default_timer()

#read track data
trackData = pd.read_csv('SmallRun7_RadarTrack.txt', sep = r'\s*,\s*', engine = 'python', low_memory = True, usecols = ['Id','TimeStamp', 'Range', 'BearingAngle', 'LateralRate', 'RangeRate', 'ModeUpdated'])

#read intensity data
intData = pd.read_csv('SmallRun7_Intensities.txt', sep = r'\s*,\s*', engine = 'python', low_memory = True, usecols = ['Timestamp', 'GroupId', 'TrackPower1', 'TrackPower2', 'TrackPower3', 'TrackPower4', 'TrackPower5', 'TrackPower6', 'TrackPower7'])


In [13]:


#initialize the dataframe that will hold the final graph data
df = pd.DataFrame()

#convert polar coords to cartesian 
#x and y values are swapped to rotate the graph 90 degrees counterclockwise (into a vertical orientation)
x = list(np.sin(np.deg2rad(trackData.BearingAngle)) * trackData.Range)
y = list(np.cos(np.deg2rad(trackData.BearingAngle)) * trackData.Range)

additionalDataDict = {
    'xVals' : x,
    'yVals' : y
}

additionalData = pd.DataFrame(additionalDataDict)

#combine the track data and final graph data frame
df = pd.concat([df, trackData, additionalData], axis = 1)

#mod lat and range rates (x and y values are flipped to rotate the graph into a vertial orientation)
vectorX = list(df.RangeRate * np.sin(np.deg2rad(df.BearingAngle)))
vectorY = list(df.RangeRate * np.cos(np.deg2rad(df.BearingAngle)) + df.LateralRate)

df.LateralRate = vectorX
df.RangeRate = vectorY

#parse intensity data
#sort df to match the intensity values
df = df.sort_values(['Id', 'TimeStamp'])

intensityData = []
columns = ['TrackPower1', 'TrackPower2', 'TrackPower3', 'TrackPower4', 'TrackPower5', 'TrackPower6', 'TrackPower7']

#iterate through the intensity data and add it to df
for i in range(10):
    temp = intData[intData.GroupId == i]
    
    if i == 9:
        intensityData = intensityData + temp['TrackPower1'].to_list()
    else:
        for col in columns:
            intensityData = intensityData + temp[col].to_list()

df['Intensity'] = intensityData

#generate some fake intensity data to test color
color = []

for i in range(len(intensityData)):
    color.append(random.randrange(-10, 40))
    
df['Color'] = color

#determine the needed animation steps
animationFrames = list(df.TimeStamp.unique())

for i in range(len(animationFrames) - 1):
    ref = animationFrames[i + 1] - animationFrames[i]
    
    if (ref < 0.020) and (ref != 0):
        animationFrames[i + 1] = animationFrames[i]

animationFrames = np.unique(animationFrames).tolist()

#scale animationFrames and add it to df
df['Frames'] = animationFrames * 64

df = df.sort_index()

#purge unnecessary data
df = df[df.Range > 0]

#create an object zero to ensure all frames are animated
filler = [0] * len(animationFrames)

objectZero = {
    'Id' : filler,
    'TimeStamp' : animationFrames,
    'Range' : filler,
    'RangeRate' : filler,
    'BearingAngle' : filler,
    'LateralRate' : filler,
    'xVals' : filler,
    'yVals' : filler,
    'Intensity' : filler,
    'Frames' : animationFrames,
    'Color' : filler
}

objZero = pd.DataFrame(objectZero)

#merge object zero and the parsed data
df = pd.concat([df, objZero], axis = 0) 

#sort df for graphing
df = df.sort_values(['TimeStamp', 'Id'])
df = df.reset_index(drop = True)


In [14]:
fig = go.Figure()

#set figure dimensions
xLen = 140 #displayed length in the x dimension in m (default is 140)
yLen = 190 #displayed length in the y dimension in m (default is 190)

fig.update_layout(width = 900, height = 1000, xaxis_range = [-xLen, xLen], yaxis_range = [-1, yLen])

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Frame:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 0, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

figaux = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = True, name = 'Velocities', hoverinfo = 'skip')
data = [go.Scatter(x = [0], y = [0], mode = 'markers',
                   marker = dict(cmax = 40, cmin = -10, showscale = False, colorscale = 'Bluered'), 
                   showlegend = True, name = 'Positions'), figaux.data[0], 
        go.Scatter(x = [0], y = [0], showlegend = False, hoverinfo = 'skip', mode = 'markers', marker = dict(cmax = 40, cmin = -10, colorbar = dict(title = 'Intensity'), colorscale = 'Bluered'))]

frames = []
for frame in animationFrames:
    
    animationData = []
    
    currentFrame = df[df.Frames == frame]
    temp = go.Scatter(x = currentFrame.xVals, y = currentFrame.yVals, mode = 'markers', marker = dict(color = currentFrame.Color), 
                      customdata = np.stack((currentFrame.Id, currentFrame.TimeStamp, currentFrame.BearingAngle, currentFrame.Range), axis=-1), 
                      hovertemplate = '<b>Target</b>: %{customdata[0]}<br>' + '<b>Time</b>: %{customdata[1]}<br>' + 
                                      '<b>Angle</b>: %{customdata[2]:,.0f}<br>' + '<b>Range</b>: %{customdata[3]}<br>' + '<extra></extra>')
    
    velocityArrows = ff.create_quiver(x = currentFrame.xVals, y = currentFrame.yVals, u = currentFrame.LateralRate, v = currentFrame.RangeRate, 
                                      scale = 4)
    
    animationData.append(temp)
    animationData.append(velocityArrows.data[0])

    frames.append(go.Frame(data = animationData, traces = [0, 1], name = frame))
    
    slider_step = {"args": [
        [frame],
        {"frame": {"duration": 0, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 0}}
    ],
        "label": frame,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

#create animation buttons (play/pause)
fig["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

fig["layout"]["sliders"] = [sliders_dict]

fig.update(data = data, frames = frames)

#style the graph
fig.update_xaxes(visible = True, showticklabels = True)
fig.update_yaxes(visible = True, showticklabels = True)

fig.update_layout(
    title_text = 'Radar Visualization',
    paper_bgcolor = 'rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)',
    #autosize = True,
    height = 900,
    width  = 1000
    )

graphColor = "black" #"#34e48c" #"PaleTurquoise"

#create guide lines
right_corner = math.cos(math.radians(45)) * 175
left_corner_x = math.cos(math.radians(45)) * -175

fig.add_shape(type = "line", x0 = 0, y0 = 0, x1 = right_corner, y1 = right_corner, line_color = graphColor)
fig.add_shape(type = "line", x0 = 0, y0 = 0, x1 = left_corner_x, y1 = right_corner, line_color = graphColor)

#add range labels to graph
for i in range(8):
    
    range_placement = np.cos(np.deg2rad(45)) * i * 25
    range_label = 25 * i
    fig.add_trace(go.Scatter(x = [-1 * range_placement], y = [range_placement], mode = "markers+text", text = range_label, textposition = "bottom left", showlegend = False, hoverinfo = 'skip', line_color = graphColor))

#create guide curves
curveAngles = [45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135]

for i in range(7):
    
    range_placement = (i + 1) * 25
    
    degree_x = range_placement * np.cos(np.deg2rad(curveAngles))
    degree_y = range_placement * np.sin(np.deg2rad(curveAngles))

    fig.add_trace(go.Scatter(x = degree_x, y = degree_y, mode = "lines", showlegend = False, hoverinfo = 'skip', line = dict(dash = 'dash'), line_color = graphColor))

#add degree labels to graph
fig.add_trace(go.Scatter(x = [0], y = [175], mode = "markers+text", text = 0, textposition = "top center", showlegend = False, hoverinfo = 'skip', line_color = graphColor))

for i in range(9):
    degree = (i + 1) * 5
    xVal = degree_x[8 - i]
    yVal = degree_y[8 - i]

    fig.add_trace(go.Scatter(x = [xVal], y = [yVal], mode = "markers+text", text = degree, textposition = "top right", showlegend = False, hoverinfo = 'skip', line_color = graphColor))
    
for i in range(9):
    degree = (i + 1) * -5
    xVal = degree_x[10 + i]
    yVal = degree_y[10 + i]

    fig.add_trace(go.Scatter(x = [xVal], y = [yVal], mode = "markers+text", text = degree, textposition = "top left", showlegend = False, hoverinfo = 'skip', line_color = graphColor))
    
#add labels (right, front, left)
fig.add_annotation(text = "Radar Front", xref= "x", yref = "y", x = 0, y = 187.5, showarrow = False)
fig.add_annotation(text = "Radar Left", xref= "x", yref = "y", x = -125, y = 187.5, showarrow = False)
fig.add_annotation(text = "Radar Right", xref= "x", yref = "y", x = 125, y = 187.5, showarrow = False)

#animation speed control (real time is 40, 40)
animationSpeed = 50 #time in ms

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = animationSpeed
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0 #speed of frame transition (should remain 0)

fig.update_xaxes(
gridcolor="black",
dtick=25,showgrid=True)

fig.update_yaxes(
gridcolor="black",
dtick=25,showgrid=True)

fig.update_layout(
    legend = dict(
        x = 0.01,
        y = 0.01,
        bgcolor = "White",
        bordercolor = "Black",
        borderwidth = 2
    )
)

fig.show()

stop = timeit.default_timer()
print('time: ', stop - start)

time:  275.1367826000205
